In [1]:
import numpy as np 
import pandas as pd 
data = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')


from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline


Understanding the data

In [2]:
print(data.shape, ";" ,test.shape, '\n')
data.head()

(891, 12) ; (418, 11) 



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
for column in data.select_dtypes(include='object').columns:
    unique_values = data[column].nunique()
    print(f"Column '{column}': {unique_values}")

Column 'Name': 891
Column 'Sex': 2
Column 'Ticket': 681
Column 'Cabin': 147
Column 'Embarked': 3


In [4]:
y = data['Survived']
X = data.drop(['Survived','PassengerId', "Name", "Ticket", "Cabin"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)



In [5]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (712, 7)
X_test shape: (179, 7)
y_train shape: (712,)
y_test shape: (179,)


In [6]:
# Count missing values per column

nan_count_per_category = X.isna().sum()

print(nan_count_per_category)

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64


In [7]:
# Count unique values for categorical columns

for column in X_test.select_dtypes(include='object').columns:
    unique_values = X_test[column].nunique()
    print(f"Column '{column}': {unique_values}")

Column 'Sex': 2
Column 'Embarked': 3


In [8]:
for column in X.select_dtypes(include='object').columns:
    unique_values = X[column].unique()
    print(f"Column '{column}': {unique_values}")

Column 'Sex': ['male' 'female']
Column 'Embarked': ['S' 'C' 'Q' nan]


Imputing missing values

In [9]:
# Impute data and perform one hot encoding

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))])

categorical_transformer= Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first'))
    
])


preprocessor = ColumnTransformer(
    transformers=[
        ('impute_category1', numerical_transformer, ['Age', 'Fare']),
        ('onehot', categorical_transformer , ['Sex', 'Embarked'])
    ],
    remainder='passthrough'
)

X_train_transformed = preprocessor.fit_transform(X_train)

print(pd.DataFrame(X_train_transformed, columns=preprocessor.get_feature_names_out(input_features=X.columns)))

     impute_category1__Age  impute_category1__Fare  onehot__Sex_male  \
0                29.745184                 15.2458               0.0   
1                31.000000                 10.5000               1.0   
2                31.000000                 37.0042               1.0   
3                20.000000                  4.0125               1.0   
4                21.000000                  7.2500               1.0   
..                     ...                     ...               ...   
707              39.000000                 83.1583               0.0   
708              19.000000                  7.8542               0.0   
709              29.745184                  7.7333               1.0   
710              36.000000                 17.4000               0.0   
711              60.000000                 39.0000               1.0   

     onehot__Embarked_Q  onehot__Embarked_S  remainder__Pclass  \
0                   0.0                 0.0                3.0   
1  

Fitting the model

In [10]:
# Find the best n estimators 
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('classifier', RandomForestClassifier())
])


param_grid = {
    'classifier__n_estimators': range(20, 201, 10)  # Range from 20 to 200 in steps of 10
}

# Create a GridSearch
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train_transformed, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)


Best Hyperparameters: {'classifier__n_estimators': 50}


In [11]:
# Fit the best model to the test set
best_model = grid_search.best_estimator_

best_model.fit(X_train_transformed, y_train)

# Predict on the test data
X_test_transformed = preprocessor.fit_transform(X_test)
y_pred = best_model.predict(X_test_transformed)


In [12]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8268156424581006


In [13]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [14]:
test_id = test['PassengerId']

x_valid = test.drop(['PassengerId', "Name", "Ticket", "Cabin"], axis=1)

x_valid_transformed = preprocessor.fit_transform(x_valid)
test_predictions = best_model.predict(x_valid_transformed)



In [15]:
print(pd.DataFrame(x_valid_transformed, columns=preprocessor.get_feature_names_out(input_features=X.columns)))

     impute_category1__Age  impute_category1__Fare  onehot__Sex_male  \
0                 34.50000                  7.8292               1.0   
1                 47.00000                  7.0000               0.0   
2                 62.00000                  9.6875               1.0   
3                 27.00000                  8.6625               1.0   
4                 22.00000                 12.2875               0.0   
..                     ...                     ...               ...   
413               30.27259                  8.0500               1.0   
414               39.00000                108.9000               0.0   
415               38.50000                  7.2500               1.0   
416               30.27259                  8.0500               1.0   
417               30.27259                 22.3583               1.0   

     onehot__Embarked_Q  onehot__Embarked_S  remainder__Pclass  \
0                   1.0                 0.0                3.0   
1  

Submitting results

In [16]:
prediction_df = pd.DataFrame({'PassengerId': test_id, 'Survived': test_predictions})


In [17]:
prediction_df.to_csv('submission.csv', index=False)